In [ ]:
!pip install -U hopsworks --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.2/170.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.1 MB/s eta 0:00:00
  Preparing metada

In [ ]:
    import hopsworks
    import pandas as pd
    from sklearn.linear_model import Ridge
    from sklearn.metrics import accuracy_score, mean_squared_error
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    import os
    import scipy
    import numpy as np

In [ ]:
import os

for k, v in os.environ.items():
    if "HOPS" in k:
        print(f'{k}={v}')

In [ ]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/189588
Connected. Call `.close()` to terminate connection gracefully.


In [73]:
# The feature view is the input set of features for your model. The features can come from different feature groups.
# You can select features from different feature groups and join them together to create a feature view
wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine",
                                  version=1,
                                  description="Read from Wine dataset",
                                  labels=["quality"],
                                  query=query)

In [ ]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (0.48s) 


In [ ]:
# Checked approaches, just for the history
# regressor = LinearRegression()
# regressor = DecisionTreeRegressor()
# regressor = RandomForestRegressor()
# regressor = KNeighborsRegressor(n_neighbors=4)
# regressor = ElasticNet(alpha=0.1, l1_ratio=0.75)

# pipe = make_pipeline(StandardScaler(), LogisticRegression(multi_class="multinomial", max_iter=150))
# pipe.fit(X_train, y_train.values.ravel())

# model = KNeighborsClassifier(n_neighbors=4)
# model.fit(X_train, y_train.values.ravel())

# Compare predictions (y_pred) with the labels in the test set (y_test)
# metrics = classification_report(y_test, y_pred, output_dict=True)
# results = confusion_matrix(y_test, y_pred)

# Create the confusion matrix as a figure, we will later store it as a PNG image file
# df_cm = pd.DataFrame(results, ['True 3', 'True 4', 'True 5', 'True 6', 'True 7', 'True 8'],
                    #  ['Pred 3', 'Pred 4', 'Pred 5', 'Pred 6', 'Pred 7', 'Pred 8'])
# cm = sns.heatmap(df_cm, annot=True)
# fig = cm.get_figure()

MSE: 0.5259271931004581
RMSE: 0.725208379088699


In [ ]:
regressor = Ridge(alpha=2, solver="auto")
regressor.fit(X_train, y_train)

Ridge(alpha=2)

In [ ]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = regressor.predict(X_test)

# calculate MSE using sklearn
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# calculate RMSE using sklearn
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

MSE: 0.5421205169230289
RMSE: 0.7362883381685662


In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'wine_model' directory will be saved to the model registry. Create the dir, first.
model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save our model to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(regressor, model_dir + "/wine_model.pkl")

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
wine_model = mr.python.create_model(
    name="wine_model",
    metrics={"accuracy" : metrics['accuracy']},
    model_schema=model_schema,
    description="Wine Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/927 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/472 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/189588/models/wine_model/1


Model(name: 'wine_model', version: 1)

In [ ]:
wine_fg = fs.get_feature_group(name="wine", version=1)
wine_df = pd.DataFrame(wine_fg.read())
wine_df['alcohol'].describe()

mu = wine_df['alcohol'].mean()
sig = wine_df['alcohol'].std()

wine_df['p_alcohol'] = wine_df['alcohol'].apply(
        lambda x: scipy.stats.norm(mu, sig).pdf(x) if x > mu else 1 - scipy.stats.norm(mu, sig).pdf(x))
wine_df

Finished: Reading data from Hopsworks, using ArrowFlight (1.07s) 


,volatile_acidity,chlorides,density,alcohol,quality,p_alcohol
0,0.40,0.071,0.99462,12.5,8,0.086973
1,0.32,0.043,0.99266,11.2,6,0.289747
2,0.30,0.046,0.99775,9.5,6,0.772731
3,0.40,0.046,0.99024,12.0,7,0.159322
4,0.45,0.080,0.99560,9.4,5,0.789869
...,...,...,...,...,...,...
5277,0.46,0.077,0.99480,10.6,6,0.336424
5278,0.47,0.076,0.99760,9.4,5,0.789869
5279,0.30,0.043,0.99296,11.0,5,0.313341
5280,0.67,0.290,0.99840,9.0,4,0.856993


In [79]:
feature_view = fs.get_feature_view(name="wine", version=1)
batch_data = feature_view.get_batch_data()
batch_data

Finished: Reading data from Hopsworks, using ArrowFlight (0.64s) 


,volatile_acidity,chlorides,density,alcohol
0,0.40,0.071,0.99462,12.5
1,0.32,0.043,0.99266,11.2
2,0.30,0.046,0.99775,9.5
3,0.40,0.046,0.99024,12.0
4,0.45,0.080,0.99560,9.4
...,...,...,...,...
5280,0.67,0.290,0.99840,9.0
5281,0.40,0.042,0.99800,9.7
5282,0.34,0.068,0.99437,10.0
5283,0.36,0.040,0.99780,10.2


In [ ]:
import random

mu = wine_df['alcohol'].mean()
sig = wine_df['alcohol'].std()

wine_df['p_alcohol'] = wine_df['alcohol'].apply(
        lambda x: scipy.stats.norm(mu, sig).pdf(x) if x > mu else 1 - scipy.stats.norm(mu, sig).pdf(x))

mu = wine_df['chlorides'].mean()
sig = wine_df['chlorides'].std()
wine_df['p_chlorides'] = wine_df['chlorides'].apply(
        lambda x: scipy.stats.norm(mu, sig).pdf(x) if x > mu else 1 - scipy.stats.norm(mu, sig).pdf(x))

mu = wine_df['volatile_acidity'].mean()
sig = wine_df['volatile_acidity'].std()
wine_df['p_volatile_acidity'] = wine_df['volatile_acidity'].apply(
        lambda x: scipy.stats.norm(mu, sig).pdf(x) if x > mu else 1 - scipy.stats.norm(mu, sig).pdf(x))

mu = wine_df['density'].mean()
sig = wine_df['density'].std()
wine_df['p_density'] = wine_df['density'].apply(
        lambda x: scipy.stats.norm(mu, sig).pdf(x) if x > mu else 1 - scipy.stats.norm(mu, sig).pdf(x))

random_quality = str(random.randint(3, 9))
filtered = wine_df.query("quality==" + random_quality).reset_index(drop=True)

print(filtered)

new_df = pd.DataFrame({"alcohol": random.choices(filtered['alcohol'], weights=filtered['p_alcohol']),
                                 "chlorides": random.choices(filtered['chlorides'], weights=abs(filtered['p_chlorides'])),
                                 "volatile_acidity": random.choices(filtered['volatile_acidity'],
                                                                    weights=abs(filtered['p_volatile_acidity'])),
                                 "density": random.choices(filtered['density'], weights=abs(filtered['p_density']))
                                 })
new_df['quality'] = random_quality
new_df

      volatile_acidity  chlorides  density  alcohol  quality  p_alcohol  \
0                0.320      0.043  0.99266     11.2        6   0.289747   
1                0.300      0.046  0.99775      9.5        6   0.772731   
2                0.260      0.071  0.99860      9.3        6   0.807093   
3                0.180      0.051  0.99530     10.6        6   0.336424   
4                0.600      0.083  0.99616      9.8        6   0.724499   
...                ...        ...      ...      ...      ...        ...   
2302             0.240      0.043  0.99378      9.9        6   0.710401   
2303             0.305      0.059  0.99331     11.5        6   0.244241   
2304             0.250      0.032  0.99189     11.5        6   0.244241   
2305             0.210      0.040  0.99006     11.3        6   0.275664   
2306             0.460      0.077  0.99480     10.6        6   0.336424   

      p_chlorides  p_volatile_acidity   p_density  
0       -9.085795           -1.348608 -109.3076

,alcohol,chlorides,volatile_acidity,density,quality
0,11.2,0.053,0.58,0.9988,6
